In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Lasso, LassoLarsCV, LassoCV, ElasticNetCV, SGDRegressor, Ridge, BayesianRidge
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split, KFold
from sklearn.metrics import mean_squared_error

In [2]:
# Read in training data
df_train = pd.read_csv('sam_data/rdk_feat_eng_whole_df_train_orig_features.csv')

# Drop the 'smiles' column 
df_train = df_train.drop(['smiles'], axis=1)

# Store gap values
Y_train = df_train.gap.values

# Delete 'gap' column
df_train = df_train.drop(['gap'], axis=1)
X_train = df_train.values
print "Train features:", X_train.shape
print "Train gap:", Y_train.shape

Train features: (999997, 282)
Train gap: (999997,)


In [3]:
# Split training data into training and validation sets as well as begin some k-fold CV
cross_X_train, cross_X_valid, cross_Y_train, cross_Y_valid = train_test_split(X_train, Y_train, test_size=0.33, random_state=42)

In [4]:
print "Training set size:", cross_X_train.shape
print "Validation set size:", cross_X_valid.shape

Training set size: (669997, 282)
Validation set size: (330000, 282)


## MODEL SELECTION

Here I'm going to work through many of the same models that I had done before (in TWK_hacking.ipynb) but with a lot more care. I'm excited to nail down some great models.

In [4]:
alphas = np.logspace(-4, 1, 30)
pca_components = [10,30,45,60,100]
num_estimators = [50,100,200]

#### LassoLarsCV

This model performs cross validation, it determines the best and most relevant alphas by itself.

In [5]:
lassoLars_clf = LassoLarsCV()
lassoLars_clf.fit(cross_X_train,cross_Y_train)
print "LassoLars best alpha: {0}".format(lassoLars_clf.alpha_)

/Users/ta24418/anaconda/lib/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 7 iterations, i.e. alpha=8.821e-05, with an active set of 7 regressors, and the smallest cholesky pivot element being 2.220e-16
  ConvergenceWarning)
/Users/ta24418/anaconda/lib/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 8 iterations, i.e. alpha=8.812e-05, with an active set of 8 regressors, and the smallest cholesky pivot element being 1.490e-08
  ConvergenceWarning)
/Users/ta24418/anaconda/lib/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 8 iterations, i.e. alpha=8.812e-05, with an active set of 8 regressors, and the smallest cholesky pivot element being 2.356e-08
  ConvergenceWarning)
/Users/ta24418/a

LassoLars best alpha: 8.77773703496e-05


In [6]:
y_pred = lassoLars_clf.predict(cross_X_valid) 
    
# Calculate RMSE and update minimum RMSE if possible
LassoLars_RMSE = np.sqrt(mean_squared_error(cross_Y_valid, y_pred))
print "LassoLars RMSE: {}".format(LassoLars_RMSE)

LassoLars RMSE: 0.305539702757


#### LassoCV

Regular LassoCV

In [7]:
lasso_clf = LassoCV()
lasso_clf.fit(cross_X_train,cross_Y_train)
print "Lasso best alpha: {}".format(lasso_clf.alpha_)

Lasso best alpha: 0.0108950693091


/Users/ta24418/anaconda/lib/python2.7/site-packages/sklearn/linear_model/coordinate_descent.py:466: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations
  ConvergenceWarning)


In [8]:
y_pred = lasso_clf.predict(cross_X_valid)

# Calculate RMSE
Lasso_RMSE = np.sqrt(mean_squared_error(cross_Y_valid, y_pred))
print "Lasso RMSE: {}".format(Lasso_RMSE)

Lasso RMSE: 0.251127479727


#### Stochastic Gradient Descent

The idea with SGD is that it minimizes empirical loss by following the path that minimizes the gradient by some learning rate (usually $\propto \alpha$). 

In [13]:
SGD_alpha = 100
SGD_RMSE = 100

for alpha in alphas:
    # Fit model and predict on validation
    sgd_clf = SGDRegressor(alpha=alpha)
    sgd_clf.fit(cross_X_train,cross_Y_train)
    y_pred = sgd_clf.predict(cross_X_valid)
    
    # Calculate RMSE and update minimum RMSE if possible
    RMSE = np.sqrt(mean_squared_error(cross_Y_valid, y_pred))
    if RMSE < SGD_RMSE:
        SGD_RMSE = RMSE
        SGD_alpha = alpha
        best_sgd = sgd_clf
print "SGD minimized with alpha: {0}, resulting in RMSE of: {1}".format(SGD_alpha,SGD_RMSE)

SGD minimized with alpha: 100, resulting in RMSE of: 100


Well, that was a disappointment...

#### Gradient Boosting Regression
Similar in nature to how SGD works. It's main identifying feature is that it fits a regression tree at each stage of the gradient minimization.

In [11]:
gradBoost_RMSE = 100

for n_estimators in num_estimators:
    print n_estimators
    gradBoost_clf = GradientBoostingRegressor(n_estimators=n_estimators)
    gradBoost_clf.fit(cross_X_train,cross_Y_train)
    y_pred = gradBoost_clf.predict(cross_X_valid)
    
    # Calculate RMSE and update minimum RMSE if possible
    RMSE = np.sqrt(mean_squared_error(cross_Y_valid, y_pred))
    if RMSE < gradBoost_RMSE:
        print RMSE
        gradBoost_RMSE = RMSE
        gradBoost_estimators = n_estimators
        best_gradBoost = gradBoost_clf
print "Gradient Boosting minimized with {0} estimators, resulting in RMSE of: {1}".format(gradBoost_estimators,gradBoost_RMSE)

50


KeyboardInterrupt: 

#### PCA + Extra Trees Regressor
Just because this is what was giving us the best performance before. I want to verify that this will rock

In [ ]:
pcaExtraTrees_RMSE = 100

for comps in pca_components:
    pca = PCA(n_components=comps)
    X_train_tr = pca.fit_transform(cross_X_train)
    X_valid_tr = pca.transform(cross_X_valid)
    
    for n_estimators in num_estimators:
        
        print comps, n_estimators
        
        extratrees_clf = ExtraTreesRegressor(n_estimators=n_estimators,n_jobs=2)
        extratrees_clf.fit(X_train_tr,cross_Y_train)
        y_pred = extratrees_clf.predict(X_valid_tr)
        
        RMSE = np.sqrt(mean_squared_error(cross_Y_valid,y_pred))
        if RMSE < pcaExtraTrees_RMSE:
            print RMSE
            pcaExtraTrees_RMSE = RMSE
            pcaExtraTrees_estimators = n_estimators
            pcaExtraTrees_components = comps
            best_pcaExtraTrees = extratrees_clf
            
print "PCA with {0} components chained ExtraTrees with {1} estimators had RMSE of {2}".format(pcaExtraTrees_components,pcaExtraTrees_estimators,pcaExtraTrees_RMSE)        

10 50
0.160239044476
10 100
0.160199866777
10 200
0.160130264909
30 50
0.158392613016
30 100
0.15829074062
30 200
0.158224597522
45 50
0.157385975105
45 100
0.157234455968
45 200
0.15717270597
60 50
60 100
60 200
0.157171712015
100 50
100 100
100

#### Ridge Regression
Inject a little regularization into this beast

In [ ]:
ridge_RMSE = 100

for alpha in alphas:
    ridge_clf = Ridge(alpha=alpha)
    ridge_clf.fit(cross_X_train,cross_Y_train)
    y_pred = ridge_clf.predict(cross_X_valid)
    
    RMSE = np.sqrt(mean_squared_error(cross_Y_valid,y_pred))
    if RMSE < ridge_RMSE:
        ridge_RMSE = RMSE
        ridge_alpha = alpha
        best_ridge = ridge_clf
        
print "Ridge RMSE: {0} with alpha: {1}".format(ridge_RMSE,ridge_alpha)

#### Support Vector Regression
I dunno if this is going to work or if it's going to be worth the effort and run-time... Just exhausting my options

In [ ]:
Cs = [0.001, 0.01, 0.1, 1]
epsilons = [0.075, 0.1, 0.125]

SVR_RMSE = 100

for comps in pca_components[:3]
    
    pca = PCA(n_components=comps)
    X_train_tr = pca.fit_transform(cross_X_train)
    X_valid_tr = pca.transform(cross_X_valid)
    
    for c in Cs:
        for eps in epsilons:
        
            svr_clf = SVR(C=c,epsilon=eps)
            svr_clf.fit(X_train_tr,cross_Y_train)
            y_pred = svr_clf.predict(X_valid_tr)
            
            RMSE = np.sqrt(mean_squared_error(cross_Y_valid,y_pred))
            if RMSE < SVR_RMSE:
                print comps, c, eps
                SVR_RMSE = RMSE
                SVR_comps = comps
                SVR_C = c
                SVR_eps = eps
                best_SVR = svr_clf
print "SVR RMSE: {0} with C: {1} epsilon {2} and under {3} components".format(SVR_RMSE,SVR_C,SVR_eps,SVR_comps)

#### ElasticNetCV

In [ ]:
l1_ratios = [.1, .5, .7, .9, .99]
elasticNet_RMSE = 100
for comps in pca_components:
    pca = PCA(n_components=comps)
    X_train_tr = pca.fit_transform(cross_X_train)
    X_valid_tr = pca.transform(cross_X_valid)
    for ratio in l1_ratios:
        print comps, ratio
        elasticNet_clf = ElasticNetCV(l1_ratio=ratio,n_jobs=3)
        elasticNet_clf.fit(X_train_tr,cross_Y_train)
        y_pred = elasticNet_clf.predict(X_valid_tr)
        
        RMSE = np.sqrt(mean_squared_error(cross_Y_valid,y_pred))
        if RMSE < elasticNet_RMSE:
            print RMSE
            elasticNet_RMSE = RMSE
            elasticNet_comps = comps
            elasticNet_l1ratio = ratio
            best_elasticNet = elasticNet_clf

print "ElasticNet RMSE: {0} with L1-ratio: {1} under {2}".format(elasticNet_RMSE,elasticNet_l1ratio,elasticNet_comps)

#### BayesianRidge

In [ ]:
alpha1s = [1e-07,1e-06,1e-05]
alpha2s = [1e-07,1e-06,1e-05]
lambda1s = [1e-07,1e-06,1e-05]
lambda2s = [1e-07,1e-06,1e-05]
bayesRidge_RMSE = 100

for alpha1 in alpha1s:
    for alpha2 in alpha2s:
        for lambda1 in lambda1s:
            for lambda2 in lambda2s:
                
                bayesRidge_clf = BayesianRidge(alpha_1=alpha1,alpha_2=alpha2,lambda_1=lambda1,lambda_2=lambda2)
                bayesRidge_clf.fit(cross_X_train,cross_Y_train)
                y_pred = bayesRidge.predict(cross_X_valid)
                
                RMSE = np.sqrt(mean_squared_error(cross_Y_valid,y_pred))
                if RMSE < bayesRidge_RMSE:
                    bayesRidge_RMSE = RMSE
                    bayesRidge_alpha1 = alpha1
                    bayesRidge_alpha2 = alpha2
                    bayesRidge_lambda1 = lambda1
                    bayesRidge_lambda2 = lambda2
                    best_bayesRidge = bayesRidge_clf
print "BayesianRidge RMSE: {}".format(bayesRidge_RMSE)
print bayesRidge_alpha1, bayesRidge_alpha2, bayesRidge_lambda1, bayesRidge_lambda2